In [48]:
import numpy
import matplotlib.pyplot

import yaml 

import wandb

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

import lightning as L
from pytorch_lightning.loggers import WandbLogger

In [7]:
# test
backbone_weights = torchvision.models.ResNet34_Weights.DEFAULT
preprocessor = backbone_weights.transforms()
backbone = torchvision.models.resnet34(weights=backbone_weights)

In [93]:
# set parameters and write to yaml file
parameters = {
    'Training': {
        'limit_train_batches': 1.0,
        'max_epochs': 10,
        'batch_size': 32,
 
    },
    'Optimizer': {
        'optimizer': 'adam',
        'weight_decay': 0.9,
        'lr': 6e-6,  
    },
    'Loss_Function': {
        'loss_function': 'qwk',
    },
    'Model': {
        'backbone': 'resnet34',
        'features': 512,
        'outputs': 3,
   },
}

parameters

{'Training': {'limit_train_batches': 1.0, 'max_epochs': 10, 'batch_size': 32},
 'Optimizer': {'optimizer': 'adam', 'weight_decay': 0.9, 'lr': 6e-06},
 'Loss_Function': {'loss_function': 'qwk'},
 'Model': {'backbone': 'resnet34', 'features': 512, 'outputs': 3}}

In [94]:
with open("parameters.yaml", 'w') as yaml_file:
    data = yaml.dump(parameters, yaml_file)

In [95]:
with open("parameters.yaml", "r") as yaml_file:
    parameters = yaml.load(yaml_file, Loader=yaml.FullLoader)
print(parameters)

{'Loss_Function': {'loss_function': 'qwk'}, 'Model': {'backbone': 'resnet34', 'features': 512, 'outputs': 3}, 'Optimizer': {'lr': 6e-06, 'optimizer': 'adam', 'weight_decay': 0.9}, 'Training': {'batch_size': 32, 'limit_train_batches': 1.0, 'max_epochs': 10}}


In [99]:
# fully-supervised fine-tuning
class Backbone(L.LightningModule):
    
    def __init__(self, n_classes, user_parameters):
        super().__init__()
        
        self.n_classes = n_classes
        self.user_parameters = user_parameters
        
        self.backbone_weights = torchvision.models.ResNet34_Weights.DEFAULT
        self.preprocessor = self.backbone_weights.transforms()
        self.backbone = torchvision.models.resnet34(weights=self.backbone_weights)
        self.n_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(self.n_features, self.n_classes)
        
        match self.user_parameters['Loss_Function']['loss_function']:
            case 'cross_entropy':
                self.loss_function = F.cross_entropy
            case 'qwk':
                from WeightedKappaLoss import WeightedKappaLoss
                self.loss_function = WeightedKappaLoss(self.n_classes, mode='quadratic')
            case _:
                self.loss_function = F.cross_entropy  # defaults to cross entropy

        self.save_hyperparameters()  # wandb
        
    def forward(self, x):
        x_processed = self.preprocessor(x)
        return self.backbone(x_processed)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.backbone.parameters(),
            lr=self.user_parameters['Optimizer']['lr'],
            weight_decay=self.user_parameters['Optimizer']['weight_decay'],
        )
        return optimizer
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        x_hat = self.forward(x)
        loss = F.cross_entropy(x_hat, y)
        self.log("train_loss", loss)  # wandb
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        x_hat = self.forward(x)
        loss = F.cross_entropy(x_hat, y)
    
    
        

In [100]:
wandb_logger = WandbLogger(
    project='debug-runs',
    save_dir='wandb-outputs',
    config=parameters,
)

In [101]:
fake_train_data = torchvision.datasets.FakeData(
    size=1000,
    image_size=(3, 512, 512),
    num_classes=3,
    transform=torchvision.transforms.ToTensor(),
)

fake_val_data = torchvision.datasets.FakeData(
    size=1000,
    image_size=(3, 512, 512),
    num_classes=3,
    transform=torchvision.transforms.ToTensor()
)

fake_train_data

Dataset FakeData
    Number of datapoints: 1000
    StandardTransform
Transform: ToTensor()

In [102]:
fake_train_dataloader = torch.utils.data.DataLoader(
    batch_size=parameters['Training']['batch_size'],
    dataset=fake_train_data,
    shuffle=True,
    num_workers=4,
)

fake_val_dataloader = torch.utils.data.DataLoader(
    batch_size=parameters['Training']['batch_size'],
    dataset=fake_val_data,
    shuffle=True,
    num_workers=4,
)

In [103]:
model = Backbone(n_classes=3, user_parameters=parameters)
trainer = L.Trainer(
    limit_train_batches=parameters['Training']['limit_train_batches'], 
    max_epochs=parameters['Training']['max_epochs'],
    logger=wandb_logger,
)

/home/felipe/Projects/multiple-instance-learning/src/WeightedKappaLoss.py:8: SyntaxWarning: invalid escape sequence '\i'
  """
/home/felipe/Projects/multiple-instance-learning/src/WeightedKappaLoss.py:27: SyntaxWarning: invalid escape sequence '\l'
  """Creates a `WeightedKappaLoss` instance.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..


In [104]:
for batch in fake_train_dataloader:
    x, y = batch[0], batch[1]
    print(x.shape, y.shape)
    break

torch.Size([32, 3, 512, 512]) torch.Size([32])


In [105]:
trainer.fit(model, fake_train_dataloader, fake_val_dataloader)
wandb.finish()

error in wandb.init()
Traceback (most recent call last):
  File "/home/felipe/Projects/multiple-instance-learning/.venv/lib/python3.12/site-packages/wandb/sdk/wandb_init.py", line 1255, in init
    wi.setup(kwargs)
  File "/home/felipe/Projects/multiple-instance-learning/.venv/lib/python3.12/site-packages/wandb/sdk/wandb_init.py", line 320, in setup
    settings._apply_init(kwargs)
  File "/home/felipe/Projects/multiple-instance-learning/.venv/lib/python3.12/site-packages/wandb/sdk/wandb_settings.py", line 1918, in _apply_init
    self.update(init_settings, source=Source.INIT)
  File "/home/felipe/Projects/multiple-instance-learning/.venv/lib/python3.12/site-packages/wandb/sdk/wandb_settings.py", line 1548, in update
    self.__dict__[key].update(settings.pop(key), source=source)
  File "/home/felipe/Projects/multiple-instance-learning/.venv/lib/python3.12/site-packages/wandb/sdk/wandb_settings.py", line 615, in update
    self._value = self._validate(self._preprocess(value))
         

UsageError: Invalid project name 'debug-runs/multiple-instance-learning': cannot contain characters '/,\\,#,?,%,:', found '/'

In [91]:
for batch in fake_train_dataloader:
    x, y = batch[0], batch[1]
    y_hat = model(x)
    acc = torch.metrics.accuracy(y_hat, y)
    print(x.shape, y.shape)
    break

Backbone(
  (preprocessor): ImageClassification(
      crop_size=[224]
      resize_size=[256]
      mean=[0.485, 0.456, 0.406]
      std=[0.229, 0.224, 0.225]
      interpolation=InterpolationMode.BILINEAR
  )
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 